In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np

In [ ]:
#idNotWork = 292590
idWork=219598

In [ ]:
geoffTrip = 401439

In [ ]:
fotos = loadOSCdata.queryOSCapi(OSCid = geoffTrip, output = 'csv', outputFile = 'photos.csv',dataType='photos')
fotos.head()

In [ ]:
acce = loadOSCdata.queryOSCapi(OSCid = idWork, output = 'csv', outputFile = 'acce.csv',dataType='accelerometer')
acce.head()